In [15]:
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import requests
import urllib.request

pd.set_option("display.max_columns", None)

In [2]:
totesurls=[]
totesurls=['https://www.thredup.com/products/handbags/totes?department_tags=handbags&search_tags=women-handbags%2Cwomen-handbags-totes']

# loop through to get url for each page
for n in range(2, 98):
    totesurls.append(f'https://www.thredup.com/products/handbags/totes?department_tags=handbags&search_tags=women-handbags%2Cwomen-handbags-totes&page={n}')
    
# check number of urls collected
len(totesurls)

97

In [8]:
### WORKS ONLY FOR THREDUP URLS ###
# helper function to get bag info and locally save bag images
def get_bag_info(url, bag_type, img_directory, counter_start, is_ethical, source):
    '''
    PARAMETERS
    ----------
    - url : str format of page url
    - bag_type : str format of bag type
    - img_directory : str format of local directory for saved images
    - counter_start : int counter start value for file naming convention
    - is_ethical : int/boolean whether source is ethical or not
    - source : str format of website source

    OUTPUT
    ----------
    - dataframe containing bag details
    - dictionary containing bag details
    - images saved locally to specified img_directory
    '''
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html5lib')

    # find all bag data in html for given url
    bags = soup.find_all('div', {'class': 'uiUj-TxKXzmIOHZu6poxM'})

    # initiate lists of data to acquire
    bag_names = []
    bag_brands = []
    bag_prices = []
    bag_urls = []
    bag_image_urls = []
    bag_img_fnames = []
    bag_descs=[]
    
    # loop through class tags to get each bag info
    for i, b in enumerate(bags):
        bag_names.append(b.find('div', {'class': '_138U7gqcrSxaloaCpyMPZg'}).text)
        bag_brands.append(b.find('h3', {'class': '_7hDjEshDdTGEs_hDVNnib'}).text)
        bag_prices.append(b.find('b').text)
        bag_urls.append(f'https://www.thredup.com'+b.find('a').attrs['href'])
    # go into each bag url and get large bag image url
    
    print(f'Getting bag image URLs')
    for i, url in enumerate(bag_urls):
        print(f'{i+1}/{len(bag_urls)}')
        soup = BeautifulSoup(requests.get(url).content, 'html.parser')
        for img in soup.findAll('img'):
            if 'large' in img.attrs['src']:
                bag_image_urls.append(img.attrs['src'])
        # get bag descriptions
        single_desc=[]
        if len(soup.find('ul', {'class':'list-disc'}).findAll('li'))>0:
            for item in soup.find('ul', {'class':'list-disc'}).findAll('li'):
                single_desc.append(item.text)
        else:
            single_desc.append('None')
        bag_descs.append(single_desc)
        time.sleep(0.25)

    # format bag data into a dictionary
    all_data = {'bag_name': bag_names,
                'brand': bag_brands,
                'price': bag_prices,
                'bag_url': bag_urls,
                'img_url': bag_image_urls,
                'img_filename': bag_img_fnames,
                'bag_desc': bag_descs}
    
    # make sure you have same number of image urls and bag names
    if len(bag_image_urls) == len(bag_names):
        # save bag images to local direcotry
        print(f'Saving images to {img_directory}.')
        for i, url in enumerate(bag_image_urls):
            brandname = bag_brands[i].strip().replace(' ', '')[:3].lower()
            fname = f'thredup_{brandname}_{counter_start+i}.png'
            file_name = f'{img_directory}/{fname}'
            urllib.request.urlretrieve(url, file_name)
            bag_img_fnames.append(fname)

        # convert dictionary to dataframe
        df = pd.DataFrame(all_data)
        df['type'] = bag_type  # adding bag type to all items in dataframe
        df['is_ethical'] = is_ethical  # boolean value (1 = yes, 0 = no)
        df['source'] = source

        print(f'Bag images have been saved to {img_directory}.')
        return df, all_data
    else:
        return None

In [68]:
# using helper function get_bag_info, loop through a portion of urls to get bag information & locally save bag image
counter = 0
df_all_totes = pd.DataFrame()
for i, url in enumerate(totesurls[:1]):
    try:
        print(f'On url: {i+1}/{len(totesurls[:1])}')
        df, dict_ = get_bag_info(url, 'Totes', 'C:/Users/Grace/Documents/goodtwin_local/img_new/new_ethical/Totes', counter, 1, 'thredup')
        counter += len(df)
        df_all_totes = df_all_totes.append(df, ignore_index=True)
    except:
        print(f'Skipping: {i+1}/{len(totesurls[:1])}')

On url: 1/1
Getting bag image URLs
1/50
2/50
3/50
4/50
5/50
6/50
7/50
8/50
9/50
10/50
11/50
12/50
13/50
14/50
15/50
16/50
17/50
18/50
19/50
20/50
21/50
22/50
23/50
24/50
25/50
26/50
27/50
28/50
29/50
30/50
31/50
32/50
33/50
34/50
35/50
36/50
37/50
38/50
39/50
40/50
41/50
42/50
43/50
44/50
45/50
46/50
47/50
48/50
49/50
50/50
Saving images to C:/Users/Grace/Documents/goodtwin_local/img_new/new_ethical/Totes.
Skipping: 1/1


""
